In [15]:
#off of wikipedia
def halton(b):
    #Generator function for Halton sequence.
    n, d = 0, 1
    while True:
        x = d - n
        if x == 1:
            n = 1
            d *= b
        else:
            y = d // b
            while x <= y:
                y //= b
            n = (b + 1) * y - x
        yield n / d

In [8]:
x = halton(5)
print(x)

<generator object halton at 0x7fbe0038f890>


In [14]:
#from tupui/halton.py on github

import numpy as np


def primes_from_2_to(n):
    """Prime number from 2 to n.
    From `StackOverflow <https://stackoverflow.com/questions/2068372>`_.
    :param int n: sup bound with ``n >= 6``.
    :return: primes in 2 <= p < n.
    :rtype: list
    """
    sieve = np.ones(n // 3 + (n % 6 == 2), dtype=np.bool)
    for i in range(1, int(n ** 0.5) // 3 + 1):
        if sieve[i]:
            k = 3 * i + 1 | 1
            sieve[k * k // 3::2 * k] = False
            sieve[k * (k - 2 * (i & 1) + 4) // 3::2 * k] = False
    return np.r_[2, 3, ((3 * np.nonzero(sieve)[0][1:] + 1) | 1)]


def van_der_corput(n_sample, base=2):
    """Van der Corput sequence.
    :param int n_sample: number of element of the sequence.
    :param int base: base of the sequence.
    :return: sequence of Van der Corput.
    :rtype: list (n_samples,)
    """
    sequence = []
    for i in range(n_sample):
        n_th_number, denom = 0., 1.
        while i > 0:
            i, remainder = divmod(i, base)
            denom *= base
            n_th_number += remainder / denom
        sequence.append(n_th_number)

    return sequence


def halton(dim, n_sample):
    """Halton sequence.
    :param int dim: dimension
    :param int n_sample: number of samples.
    :return: sequence of Halton.
    :rtype: array_like (n_samples, n_features)
    """
    big_number = 10
    while 'Not enought primes':
        base = primes_from_2_to(big_number)[:dim]
        if len(base) == dim:
            break
        big_number += 1000

    # Generate a sample using a Van der Corput sequence per dimension.
    sample = [van_der_corput(n_sample + 1, dim) for dim in base]
    sample = np.stack(sample, axis=-1)[1:]

    return sample


print(van_der_corput(10))
# [0.0, 0.5, 0.25, 0.75, 0.125, 0.625, 0.375, 0.875, 0.0625, 0.5625]
print(halton(67, 21)[1])
print(len(halton(67,21)))
# [[ 0.5         0.33333333]
#  [ 0.25        0.66666667]
#  [ 0.75        0.11111111]
#  [ 0.125       0.44444444]
#  [ 0.625       0.77777778]]

[0.0, 0.5, 0.25, 0.75, 0.125, 0.625, 0.375, 0.875, 0.0625, 0.5625]
[0.25       0.66666667 0.4        0.28571429 0.18181818 0.15384615
 0.11764706 0.10526316 0.08695652 0.06896552 0.06451613 0.05405405
 0.04878049 0.04651163 0.04255319 0.03773585 0.03389831 0.03278689
 0.02985075 0.02816901 0.02739726 0.02531646 0.02409639 0.02247191
 0.02061856 0.01980198 0.01941748 0.01869159 0.01834862 0.01769912
 0.01574803 0.01526718 0.01459854 0.01438849 0.01342282 0.01324503
 0.01273885 0.01226994 0.01197605 0.01156069 0.01117318 0.01104972
 0.0104712  0.01036269 0.01015228 0.01005025 0.00947867 0.00896861
 0.00881057 0.00873362 0.00858369 0.0083682  0.00829876 0.00796813
 0.0077821  0.00760456 0.00743494 0.00738007 0.00722022 0.00711744
 0.00706714 0.00682594 0.00651466 0.00643087 0.00638978 0.00630915
 0.0060423 ]
21
